### Neural Networks

Neural networks are computational models inspired by the human brain, designed to recognize patterns and
make decisions based on data. They consist of interconnected layers of nodes, or "neurons," which process
and transform input information. Through training, neural networks learn to improve their accuracy in tasks like image recognition, language processing, and more.Neural networks comprise of layers that perform operations on data.

In [1]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### Dataset
The Pima Indians Diabetes dataset is a popular dataset in the field of machine learning and statistics, particularly for those working on classification problems. 

Dataset Overview:
**Source**: The dataset was created by the National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK) and is available in the UCI Machine Learning Repository.
**Purpose**: The dataset is used to predict the onset of diabetes within five years based on diagnostic measures.
**Features**: The dataset contains 768 samples, each with 8 features. 

The features are:

1. Pregnancies: Number of times pregnant.
2. Glucose: Plasma glucose concentration (mg/dL) a 2 hours in an oral glucose tolerance test.
3. Blood Pressure: Diastolic blood pressure (mm Hg) at the time of screening.
4. Skin Thickness: Triceps skinfold thickness (mm) measured at the back of the upper arm.
5. Insulin: 2-Hour serum insulin (mu U/ml).
6. BMI: Body mass index (weight in kg/(height in m)^2).
7. Diabetes Pedigree Function: A function that scores likelihood of diabetes based on family history.
8. Age: Age of the individual (years).

**Outcome**: Whether or not the individual has diabetes (1 for positive, 0 for negative).

In [4]:
!head /scratch/vp91/$USER/intro-to-pytorch/data/pima-indians-diabetes.data.csv

6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
5,116,74,0,0,25.6,0.201,30,0
3,78,50,32,88,31.0,0.248,26,1
10,115,0,0,0,35.3,0.134,29,0
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0.0,0.232,54,1


In [5]:
datapath = os.path.expandvars('/scratch/vp91/$USER/intro-to-pytorch/data/pima-indians-diabetes.data.csv')
print(datapath)

/scratch/vp91/jxj900/intro-to-pytorch/data/pima-indians-diabetes.data.csv


### Curate the dataset
Load the dataset, split into features (X) and output (y) variables

In [6]:
dataset = np.loadtxt(datapath, delimiter=',')
X = dataset[:,0:8] 
y = dataset[:,8]

### Convert the data to tensors

In [7]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

### Defining the Model

When designing the model, keep the following points in mind:

1. The input features in the input layer must match the input features in the dataset (`X_tensor`).
2. A high number of layers can increase computation time, while too few layers may result in poor predictions.
3. Each layer should be followed by an activation function.

In this example, we will use a 3-layer neural network:

1. The input layer expects 8 features.
2. The first hidden layer has 12 neurons, followed by a ReLU activation function.
3. The second hidden layer has 8 neurons, followed by another ReLU activation function.
4. The output layer has one neuron, followed by a sigmoid activation function.

The sigmoid function outputs values between 0 and 1, which is exactly what we need.


In PyTorch, neural networks can be defined using different approaches, and two common ones are the Sequential model and the class-based model.

#### Sequential model

* The Sequential model is a simple, linear stack of layers where each layer has a single input and output. It is useful for straightforward feedforward networks where layers are applied in a sequential order.
* It is easier to use for simple architectures where layers are applied in a linear fashion.
* Defined Using: *torch.nn.Sequential*.

In [10]:
seq_model = nn.Sequential(

    # First hidden layer
    # This layer that takes an input of size 8 and outputs 12 neurons. 
    # The input is a vector of size 8, and the output is a vector 
    # of size 12. The layer applies a linear transformation to the input data.
    nn.Linear(8, 12),

    #ReLU (Rectified Linear Unit) activation function
    nn.ReLU(),

    # Second hidden layer
    #This layer takes the 12 neurons from the previous layer as input and outputs a 
    # vector of size 8
    nn.Linear(12, 8),

    #ReLU (Rectified Linear Unit) activation function
    nn.ReLU(),

    # Output hidden layer
    # Takes the 8 neurons as input and outputs a single value 
    # (since the output dimension is 1).
    nn.Linear(8, 1),

    #Sigmoid activation function
    nn.Sigmoid()
)

In [11]:
print(seq_model)

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)


### Class-Based Model

The class-based model allows you to define a network by subclassing torch.nn.Module. This approach provides greater flexibility and control, making it suitable for complex models and custom behaviors.

* Offers full control over the network architecture, including complex data flows, multiple inputs/outputs, and custom forward methods.
* Custom Forward Pass: You can define complex forward passes and control data flow through the network.
* Dynamic Behavior: Allows for dynamic computations, such as conditional layers or operations.
* Defined Using: Subclass of torch.nn.Module

In [12]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

In [13]:
class_model = PimaClassifier()
print(class_model)

PimaClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


### Define the loss function
Binary Cross-Entropy (BCE) Loss: Measures the performance of a classification model whose output is a probability value between 0 and 1. It calculates the difference between the predicted probabilities and the actual binary labels (0 or 1) and penalizes the model more when the predictions are further from the true labels.

BCELoss(y', y)=−[ylog(y')+(1−y)log(1−y')]

Where, y' is the predicted output and y is the actual otput.

In [14]:
loss_fn = nn.BCELoss()

### Optimizer

Optimizer's main role is to update the model's parameters based on the gradients computed during backpropagation.

1. **Parameter Updates**: Optimizers adjust the weights and biases of the neural network to reduce the loss. This involves applying algorithms that modify the parameters to minimize the difference between the predicted outputs and the actual targets.
2. **Learning Rate Management**: Most optimizers include mechanisms to adjust the learning rate, either statically or dynamically, to control how large the parameter updates are.

In this example we use an optimizer called Adaptive Moment Estimation (Adam). This computes an adaptive learning rates for each parameter by considering both the mean and the variance of the gradients.

In [15]:
optimizer = optim.Adam(class_model.parameters(), lr=0.001)

#### Training the Model

Training a neural network involves epochs and batches, which define how data is fed to the model:

- **Epoch:** A full pass through the entire training dataset.
- **Batch:** A subset of samples processed at a time, with gradient descent performed after each batch.

In practice, the dataset is divided into batches, and each batch is processed sequentially in a training loop. Completing all batches constitutes one epoch. The process is repeated for multiple epochs to refine the model.

Batch size is constrained by system memory (GPU memory), and computational demands scale with batch size. More epochs and batches lead to better model performance but increase training time. The optimal number of epochs and batch size is often determined through experimentation.

#### Purpose of optimizer.zero_grad(), loss.backward(), optimizer.step()

**optimizer.zero_grad()**: During training, gradients accumulate by default in PyTorch. This means that if you don’t clear them, gradients from multiple backward passes (from different batches) will be added together, which can lead to incorrect updates to the model parameters.
By calling optimizer.zero_grad(), you ensure that gradients from previous steps are reset to zero, preventing them from affecting the current update.

**loss.backward()**:  Calculates the gradients of the loss with respect to each parameter of the model. This is done using backpropagation, a key algorithm for training neural networks.

**optimizer.step()**: Used to update the model's parameters based on the gradients computed during during the backward pass (**loss.backward()**).

In [16]:
%%time
n_epochs = 100
batch_size = 10
 
for epoch in range(n_epochs):
    for i in range(0, len(X_tensor), batch_size):

        # For each mini-batch, the input data (Xbatch) is transferred to the GPU (device), 
        # allowing the computations to run on the GPU for faster training.
        Xbatch = X_tensor[i:i+batch_size]

        # The model takes the input mini-batch (Xbatch) and makes predictions (y_pred).
        y_pred = class_model(Xbatch)

        # The corresponding expected output labels for the current mini-batch are also 
        # transferred to the GPU.
        ybatch = y_tensor[i:i+batch_size]

        # The loss function calculates the error (loss) between the predicted values (y_pred) 
        # and the actual labels (ybatch).
        loss = loss_fn(y_pred, ybatch)

        # Before backpropagation, the gradients of the model parameters are reset to zero. 
        # This ensures that the gradients from the previous iteration do not accumulate.
        optimizer.zero_grad()

        # Backpropagation is performed to compute the gradients of the loss with respect to 
        # the model parameters.
        loss.backward()

        # The optimizer updates the model's parameters using the computed gradients.
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 1.0133854150772095
Finished epoch 1, latest loss 0.7441348433494568
Finished epoch 2, latest loss 0.6407996416091919
Finished epoch 3, latest loss 0.5694960951805115
Finished epoch 4, latest loss 0.5475271940231323
Finished epoch 5, latest loss 0.5354700088500977
Finished epoch 6, latest loss 0.523318350315094
Finished epoch 7, latest loss 0.5216410756111145
Finished epoch 8, latest loss 0.5160097479820251
Finished epoch 9, latest loss 0.5094536542892456
Finished epoch 10, latest loss 0.5070609450340271
Finished epoch 11, latest loss 0.5028208494186401
Finished epoch 12, latest loss 0.49739667773246765
Finished epoch 13, latest loss 0.4959896504878998
Finished epoch 14, latest loss 0.48979130387306213
Finished epoch 15, latest loss 0.48862120509147644
Finished epoch 16, latest loss 0.48711657524108887
Finished epoch 17, latest loss 0.48717817664146423
Finished epoch 18, latest loss 0.48560628294944763
Finished epoch 19, latest loss 0.48733898997306824
Fini

# Evaluate the Model

Currently, we are testing the model on the training dataset. Ideally, we should split the data into separate training and testing datasets, or use a distinct dataset for evaluation. For simplicity, we are testing the model on the same data used for training.



In [17]:
with torch.no_grad():
    y_pred = class_model(X_tensor)
 
accuracy = (y_pred.round() == y_tensor).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.7447916865348816


### Exercise

1. **Increase the number of layers in the neural network.** Observe any changes in accuracy.
2. **Change the optimizer from Adam to [Stochastic Gradient Descent (SGD)](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html).** Evaluate how this affects the loss calculation.